# **Input and Output Ports of the Manipulation Station**


## Notebook Setup 
The following cell will install Drake, checkout the manipulation repository, and set up the path (only if necessary).
- On Google's Colaboratory, this **will take approximately two minutes** on the first time it runs (to provision the machine), but should only need to reinstall once every 12 hours.  

More details are available [here](http://manipulation.mit.edu/drake.html).

In [1]:
import importlib
import sys
from urllib.request import urlretrieve

server_args = []
if 'google.colab' in sys.modules and importlib.util.find_spec('manipulation') is None:
    urlretrieve(f"http://manipulation.csail.mit.edu/scripts/setup/setup_manipulation_colab.py",
                "setup_manipulation_colab.py")
    from setup_manipulation_colab import setup_manipulation
    setup_manipulation(manipulation_sha='c1bdae733682f8a390f848bc6cb0dbbf9ea98602', drake_version='0.25.0', drake_build='releases')
    server_args = ['--ngrok_http_tunnel']
    

Cloning into '/opt/manipulation'...

HEAD is now at c1bdae7 make running_as_notebook a util, and add colab testing logic

/usr/lib/python3.7/runpy.py:125: RuntimeWarning: 'notebook.nbextensions' found in sys.modules after import of package 'notebook', but prior to execution of 'notebook.nbextensions'; this may result in unpredictable behaviour
  warn(RuntimeWarning(msg))
Enabling notebook extension jupyter-js-widgets/extension...
Paths used for configuration of notebook: 
    	/usr/etc/jupyter/nbconfig/notebook.json
      - Validating: OK
Paths used for configuration of notebook: 
    	/usr/etc/jupyter/nbconfig/notebook.json
/sbin/ldconfig.real: /usr/local/lib/python3.7/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK

Reason for being yanked: Name generation process created inappropriate id values
ERROR: nbclient 0.5.3 has requirement jupyter-client>=6.1.5, but you'll have jupyter-clien

In [2]:
# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)

import numpy as np

## Access System Input/Output Values
In this exercise, you will explore the [ManipulationStation](https://drake.mit.edu/pydrake/pydrake.examples.manipulation_station.html?highlight=manipulationstation#pydrake.examples.manipulation_station.ManipulationStation) that was mentioned during the lecture. The system diagram is shown below. 
<img src="https://raw.githubusercontent.com/RussTedrake/manipulation/master/figures/exercises/manipulation_station_diagram.png" width="1000">

You should recall that the orange ports are the ones that do not exist for the actual hardware platform.


Now we construct a ManipulationStation object and finalize the system setting. To get a sense of what this manipulation station looks like, you can open the meshcat viewer from the generated link as usual. There should be a bookshelf and a Kuka arm with a gripper attached (it might take a few seconds to load).

In [3]:
from pydrake.examples.manipulation_station import ManipulationStation
from pydrake.systems.framework import DiagramBuilder
from pydrake.systems.meshcat_visualizer import MeshcatVisualizer
from pydrake.systems.analysis import Simulator

station = ManipulationStation()
station.SetupManipulationClassStation()
plant = station.get_mutable_multibody_plant()
station.Finalize()

builder = DiagramBuilder()
builder.AddSystem(station)
scene_graph = station.get_mutable_scene_graph()
meshcat = MeshcatVisualizer(scene_graph,
            zmq_url=zmq_url)
builder.AddSystem(meshcat)
builder.Connect(station.GetOutputPort("query_object"),
        meshcat.get_geometry_query_input_port())
diagram = builder.Build()
simulator = Simulator(diagram)
simulator.Initialize()

Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6000...
You can open the visualizer by visiting the following URL:
http://99ccf201b68f.ngrok.io/static/
Connected to meshcat-server.


[**Context**](https://drake.mit.edu/pydrake/pydrake.systems.framework.html?highlight=context#pydrake.systems.framework.Context_) is an abstract class template that represents all the typed values that are used in a System’s computations: time, numeric-valued input ports, numerical state, and numerical parameters. There are also type-erased abstract state variables, abstract-valued input ports, abstract parameters, and a double accuracy setting. It is important to note that a **Context** is designed to be used only with the System that created it. State and Parameter data can be copied between contexts for compatible systems as necessary. One of the most common mistakes is to pass the wrong context. Although most methods in drake should throw an error if you pass a context from the wrong system, but not all of them do yet. 

In the cell below, we first create a root context from the diagram, and then we retrieve the contexts of the subsystems from the root context.

In [4]:
# initialize context
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)
station_context = station.GetMyContextFromRoot(context)

In this exercise, you will familiarize yourself with the input and output mechanism from the manipulation station system. Remember you can always generate a schematic view of your system by running the cell below. By clicking the "+" sign on the manipulation_station, you can get a more detailed view of the diverse modules within the manipulation station.

In [5]:
from IPython.display import HTML
from pydrake.systems.framework import GenerateHtml

diagram.set_name("diagram")
HTML('<script src="https://unpkg.com/gojs/release/go.js"></script>' + GenerateHtml(diagram))

Now if we set the joint position of the Kuka arm, we should expect to get the same values from the iiwa_position_measured port, which can be found from the output ports of **manipulation_station** object. Below we demonstrate how this can be done using **drake**'s syntax. You may also find it useful to review the **system** class documentation [here](https://drake.mit.edu/pydrake/pydrake.systems.framework.html?highlight=output_port#pydrake.systems.framework.System_).

In [6]:
# provide initial states
q0 = np.array([-1.57, 0.1, 0, -1.2, 0, 1.6, 0])
# set the joint positions of the kuka arm
station.SetIiwaPosition(station_context, q0)
# examine the output port
station.GetOutputPort("iiwa_position_measured").Eval(station_context)

array([-1.57,  0.1 ,  0.  , -1.2 ,  0.  ,  1.6 ,  0.  ])

Note that the [output port](https://drake.mit.edu/pydrake/pydrake.systems.framework.html?highlight=outputport#pydrake.systems.framework.OutputPort) named "iiwa_position_measured" is first retrieved from the station and then evaluated using **Eval** method. This is a very common approach to read the values of a selected output port.

Alternatively, you may retrieve the joint angles from the **plant**, which is a subsystem of the manipulation station.

In [7]:
joint_angles = []
for i in range(1, 8):
  joint_angles.append(
  plant.GetJointByName('iiwa_joint_{}'.format(i)).get_angle(plant_context)
  )

# alternatively, use GetPositions to obtain the generalized positions
# from the plant context
q_general = plant.GetPositions(plant_context, 
                               plant.GetModelInstanceByName("iiwa"))

print(joint_angles)
print(q_general)

[-1.57, 0.1, 0.0, -1.2, 0.0, 1.6, 0.0]
[-1.57  0.1   0.   -1.2   0.    1.6   0.  ]


# Exercise a: Code Submission
Now, it's your time to code! Use **GetOutputPort** and **Eval** to retrieve the joint velocities from the "iiwa_velocity_estimated" output port. Note that we have set the velocities for you. 

In [8]:
station.SetIiwaVelocity(station_context, np.zeros(7,))

Below, `get_velocity(station, station_context)` is the function you must modify to query values from "iiwa_velocity_estimated".

In [12]:
def get_velocity(station, station_context):
  velocity_estimated = station.GetOutputPort("iiwa_velocity_estimated").Eval(station_context)
  return velocity_estimated

You can check if you got the implementation correct by running the below autograder.

In [13]:
from manipulation.exercises.robot.test_manipulation_io import TestManipulationIO
from manipulation.exercises.grader import Grader

Grader.grade_output([TestManipulationIO], [locals()], 'results.json')
Grader.print_test_results('results.json')


Total score is 5/5.

Score for Test get_velocity implementation is 5/5.


"You may also command the Kuka joint positions via the [FixValue](https://drake.mit.edu/pydrake/pydrake.systems.framework.html?highlight=fixvalue#pydrake.systems.framework.InputPort) method. It fixes the inputs to the input port with selected values. An example is given below.  

In [14]:
q_command = np.array([-1.57, 0.5, 0, -1.2, 0, 1.6, 0])
station.GetInputPort('iiwa_position').FixValue(station_context, q_command)

Now you should confirm by **GetOutputPort** and **Eval** that the "iiwa_position_commanded" port returns the same value as the q_command.

In [15]:
station.GetOutputPort("iiwa_position_commanded").Eval(station_context)

array([-1.57,  0.5 ,  0.  , -1.2 ,  0.  ,  1.6 ,  0.  ])

Please note that the *iiwa_position_commanded* and the *iiwa position* are NOT the same variable. The *iiwa_position_commanded* are the commanded positions sent to the robot, whereas the *iiwa_positions* are the current positions of the simulated robot. The input and output match perfectly in this case because of the luxury of simulation. However, the same observation can not be made from the feedforward torque (system input) and the commanded torque (system output). Next, you will investigate why. First, let us provide a zero feedforward torque to the "iiwa_feedforward_torque" port.

In [16]:
station.GetInputPort("iiwa_feedforward_torque").FixValue(station_context, np.zeros(7,))
tau_no_ff = station.GetOutputPort("iiwa_torque_commanded").Eval(station_context)
print('feedforward torque: {}'.format(np.zeros(7,)))
print('commanded torque with no feedforward torque:{}'.format(tau_no_ff))

feedforward torque: [0. 0. 0. 0. 0. 0. 0.]
commanded torque with no feedforward torque:[-13.59469518  83.99514279 -13.07262911 -34.27181406 -13.01727483
  13.33800207   0.19496751]


Now try the same experiment with a non-zero feedforward torque as shown below.

In [20]:
tau_ff = np.linspace(3.1, 3.7, 7)
print('feedforward torque: {}'.format(tau_ff))
station.GetInputPort("iiwa_feedforward_torque").FixValue(station_context, tau_ff)
torque_commanded = station.GetOutputPort("iiwa_torque_commanded").Eval(station_context)
print('the commanded torque: {}'.format(torque_commanded))

feedforward torque: [3.1 3.2 3.3 3.4 3.5 3.6 3.7]
the commanded torque: [-10.49469518  87.19514279  -9.77262911 -30.87181406  -9.51727483
  16.93800207   3.89496751]


# Exercise b: Written Problem.
Below, we have a question for you.

**In this exercise, please explain what causes the discrepancy between the feedforward torque and the commanded torque.**

HINT: can you find any relationship among *tau_ff*, *tau_no_ff*, *torque_commanded*?

## Your Answer

The initial commanded torque with no feedforward torque is a non-zero constant. Hence, comanded torque with no feedforward torque is feedforward torque plus some constant. The reason is that the robot arm itself has weight, so we need a initial torque for each joint to cover the arm's weight. Otherwise, the robot arm would "fall down". The initial torque that covers the arm's weight is constant because each joint needs the same torque to cover the weight.

## How will this notebook be graded?

If you are enrolled in the class, this notebook will be graded using [Gradescope](www.gradescope.com). You should have gotten the enrollment code on our announcement in Piazza.

For submission of this assignment, you must do wo things. 
- Download and submit this notebook `manipulation_station_io.ipynb` to Gradescope's notebook submission section, along with your notebook for the first problem.
- Copy and Paste your answer in the qualitative problem to Gradescope's written submission section. 

We will evaluate the local functions in the notebook to see if the function behaves as we have expected. For this exercise, the rubric is as follows:
- [5pts] `get_velocity` must be implemented correctly. 
- [5pts] You must answer correctly why there is a difference between feed-forward torque and commanded torque. 

## Additional Note.

So far we have not looked into the outputs of simulated camera sensors. Fortunately, accessing the camera data can be done in an almost exactly the same way as we have shown above. We will get to it soon! 